### Import dependecies

In [1]:
import tensorflow as tf
from functools import partial
import sys
sys.path.insert(0, "tools")
from preprocess import X_train, y_train, X_valid, y_valid

### Define Fully connected convulltional neural network architecture

In [3]:
#Initial hyperparameters
BATCH_SIZE = 32
NUM_EPOCHS = 100
OPTIMIZER = tf.keras.optimizers.Adam(learning_rate=1e-4)
EARLY_STOPPING = tf.keras.callbacks.EarlyStopping(monitor='val_loss', mode='min', patience=10)

In [4]:
#Wrap layers for convinient use
Conv2D = partial(tf.keras.layers.Conv2D, kernel_size = (3,3), padding = "SAME", activation = 'relu')
MaxPool2D = partial(tf.keras.layers.MaxPool2D, pool_size = (2,2), strides = (2,2))


In [5]:
from tensorflow.keras.applications.vgg16 import VGG16

vgg = VGG16(weights="imagenet", include_top=False,
	input_tensor=tf.keras.layers.Input(shape=(224, 224, 3)))

vgg.trainable = False
flatten = vgg.output
flatten = tf.keras.layers.Flatten()(flatten)
bboxHead = tf.keras.layers.Dense(128, activation="relu")(flatten)
bboxHead = tf.keras.layers.Dense(64, activation="relu")(bboxHead)
bboxHead = tf.keras.layers.Dense(32, activation="relu")(bboxHead)
bboxHead = tf.keras.layers.Dense(4, activation="sigmoid")(bboxHead)
model = tf.keras.models.Model(inputs=vgg.input, outputs=bboxHead)


In [ ]:
'''model = tf.keras.Sequential([
    Conv2D(filters = 64, input_shape = (224, 224, 3)),
    Conv2D(filters = 64),
    MaxPool2D(),
    Conv2D(filters = 128), 
    Conv2D(filters = 128), 
    MaxPool2D(),
    Conv2D(filters = 256), 
    Conv2D(filters = 256),
    Conv2D(filters = 256),
    MaxPool2D(),
    Conv2D(filters = 512),
    Conv2D(filters = 512),
    Conv2D(filters = 512),
    MaxPool2D(),
    Conv2D(filters = 512),
    Conv2D(filters = 512),
    Conv2D(filters = 512),
    MaxPool2D(), 
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation= 'relu'),
    tf.keras.layers.Dense(64, activation = 'relu'),
    tf.keras.layers.Dense(32, activation = 'relu'),
    tf.keras.layers.Dense(4, activation = 'softmax')

])'''

In [7]:
#Compiling the model
model.compile(loss = "mse", optimizer=OPTIMIZER)

In [8]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 640, 640, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 640, 640, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 640, 640, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 320, 320, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 320, 320, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 320, 320, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 160, 160, 128)     0     

In [9]:
history = model.fit(X_train, y_train, validation_data=(X_valid, y_valid), epochs=NUM_EPOCHS, batch_size=BATCH_SIZE)

Epoch 1/100


ResourceExhaustedError: Graph execution error:

Detected at node 'model/block1_conv1/Relu' defined at (most recent call last):
    File "c:\Users\sobsd\miniconda3\envs\ANPR\lib\runpy.py", line 197, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "c:\Users\sobsd\miniconda3\envs\ANPR\lib\runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "c:\Users\sobsd\miniconda3\envs\ANPR\lib\site-packages\ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "c:\Users\sobsd\miniconda3\envs\ANPR\lib\site-packages\traitlets\config\application.py", line 1043, in launch_instance
      app.start()
    File "c:\Users\sobsd\miniconda3\envs\ANPR\lib\site-packages\ipykernel\kernelapp.py", line 712, in start
      self.io_loop.start()
    File "c:\Users\sobsd\miniconda3\envs\ANPR\lib\site-packages\tornado\platform\asyncio.py", line 215, in start
      self.asyncio_loop.run_forever()
    File "c:\Users\sobsd\miniconda3\envs\ANP

### Conver h5 model into onnx 

In [8]:
import tf2onnx 
onnx_model, _ = tf2onnx.convert.from_keras(model)

with open('models/detectionv5.onnx', 'wb') as f:
    f.write(onnx_model.SerializeToString())

In [ ]:

import visualkeras
from PIL import ImageFont
visualkeras.layered_view(model, legend=True)

In [ ]:
import matplotlib.pyplot as plt 
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()